# CoE 197 Z Machine Exercise 5: Vehicle Counter
John Eric Estrada <br>
2022 - 02217

In [1]:
%pip install gradio
%pip install ultralytics
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gradio as gr
import cv2
import numpy as np
from ultralytics import YOLO
import torch

model = YOLO("yolo11x.pt")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

print(device)

/Users/tobi/Documents/coe197z/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu


In [3]:
def detect_vehicles(image):
    # Convert to numpy array if it's not already
    if isinstance(image, np.ndarray):
        img = image.copy()
    else:
        img = np.array(image)
    
    # Convert RGB to BGR if needed
    if img.shape[2] == 3:  # RGB image
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    # Run YOLOv8 detection
    results = model(img)
    
    # Vehicle classes to count (car, truck, bus, motorcycle, bicycle)
    vehicle_classes = [2, 3, 5, 6, 7]
    
    # Initialize vehicle count
    vehicle_count = 0
    
    # Create a copy of the image for drawing boxes
    img_with_boxes = img.copy()
    
    # Process detections
    for result in results:
        for box in result.boxes:
            # Check if detection is a vehicle
            if int(box.cls) in vehicle_classes:
                vehicle_count += 1
                # Get box coordinates
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                # Draw bounding box (green color)
                cv2.rectangle(img_with_boxes, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Convert back to RGB for display
    img_with_boxes = cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB)
    
    return vehicle_count, img_with_boxes

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# CoE 197Z ME5: Vehicle Counter")
    gr.Markdown("John Eric Estrada")
    
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(label="Input Image", type="numpy")
            submit_btn = gr.Button("Detect Vehicles")
        with gr.Column():
            count_output = gr.Number(label="Number of Vehicles")
            image_output = gr.Image(label="Detected Vehicles")
    
    submit_btn.click(
        fn=detect_vehicles,
        inputs=image_input,
        outputs=[count_output, image_output]
    )

In [ ]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.



0: 448x640 10 persons, 8 cars, 7 motorcycles, 2 buss, 10 trucks, 1 handbag, 715.3ms
Speed: 2.5ms preprocess, 715.3ms inference, 1.9ms postprocess per image at shape (1, 3, 448, 640)
